# DEEP LEARNING PROJECT 1
---

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/deep

/content/drive/My Drive/deep


In [0]:
import helpers
import torch.nn as nn
import numpy as np
import config
from models.FCN import FCN
from models.SiameseNet import SiameseNet
from train import train_siamese
from predict import predict_siamese
import torch.utils.data as data
import matplotlib.pyplot as plt
from datasets import PairDataset
import torch

In [0]:
rounds = 10

In [0]:
pairs = helpers.generate_pair_sets(config.NB_SAMPLES)

train_dataset = PairDataset(pairs[0], pairs[1], pairs[2])
train_dataloader = data.DataLoader(dataset=train_dataset, batch_size=config.TRAIN_BATCH_SIZE, shuffle=True)

test_dataset = PairDataset(pairs[3], pairs[4], pairs[5])
test_dataloader = data.DataLoader(dataset=test_dataset, batch_size=config.TEST_BATCH_SIZE, shuffle=True)

# FCNEURONS (inner network) search

In [6]:
cuda = torch.cuda.is_available()
if cuda:
    print("CUDA available")
else:
    print("NO CUDA")

round_results_nb_neurons_inner = [] #3D

for i in range(rounds):
    results = [] #training_losses, training_acc, test_losses, test_acc
    
    print('round {0} start'.format(i+1))
    
    for ind1, nb in enumerate(config.NB_LAYERS):
        results_tmp = []
        
        for ind2, n in enumerate(config.FCNEURONS):
        
            subnet = FCN(nb_hidden_layers=nb, hidden_layer = n)

            model = SiameseNet(subnet)

            training_losses, training_acc, _, _, test_losses, test_acc, _, _ = train_siamese(
                model = model, dataloader = train_dataloader, test_dataloader = test_dataloader,
                epochs = config.EPOCHS, learning_rate = config.LEARNING_RATE, aux_loss = True,
                alpha = config.ALPHA)

            print('{0}/{1}'.format(ind1 * len(config.FCNEURONS) + ind2 + 1, len(config.NB_LAYERS) * len(config.FCNEURONS)))
            print('With parameters nb_hid_inner/hid_inner/nb_hid_out/hid_outer/alpha : {0}/{1}/{2}/{3}/{4}'.format(nb,
                                                                                                                   n,
                                                                                                                   config.SIAMESE_NET_NB_HIDDEN,
                                                                                                                   config.SIAMESE_NET_HIDDEN_LAYER,
                                                                                                                   config.ALPHA))
            final_test_loss, final_test_loss_acc = test_losses[-1], test_acc[-1]
            print("On the test set we obtain a loss of {:.2f} and an accuracy of {:.2f}".format(final_test_loss,final_test_loss_acc))

            results_tmp.append([training_losses, training_acc, test_losses, test_acc])
    
        results.append(results_tmp)
    
    print('round {0} end'.format(i+1))
    round_results_nb_neurons_inner.append(results)

CUDA available
round 1 start
1/10
With parameters nb_hid_inner/hid_inner/nb_hid_out/hid_outer/alpha : 1/32/2/128/0.5
On the test set we obtain a loss of 0.46 and an accuracy of 0.87
2/10
With parameters nb_hid_inner/hid_inner/nb_hid_out/hid_outer/alpha : 1/64/2/128/0.5
On the test set we obtain a loss of 0.38 and an accuracy of 0.88
3/10
With parameters nb_hid_inner/hid_inner/nb_hid_out/hid_outer/alpha : 1/128/2/128/0.5
On the test set we obtain a loss of 0.42 and an accuracy of 0.88
4/10
With parameters nb_hid_inner/hid_inner/nb_hid_out/hid_outer/alpha : 1/256/2/128/0.5
On the test set we obtain a loss of 0.49 and an accuracy of 0.88
5/10
With parameters nb_hid_inner/hid_inner/nb_hid_out/hid_outer/alpha : 1/512/2/128/0.5
On the test set we obtain a loss of 0.55 and an accuracy of 0.87
6/10
With parameters nb_hid_inner/hid_inner/nb_hid_out/hid_outer/alpha : 2/32/2/128/0.5
On the test set we obtain a loss of 0.38 and an accuracy of 0.90
7/10
With parameters nb_hid_inner/hid_inner/nb_hid

In [7]:
np.savez("results-fcn-nb_fcneurons_inner_search",round_results_nb_neurons_inner)

copy_of = np.array(round_results_nb_neurons_inner).copy()

last_accs_only = copy_of[:, :, :, 3, config.EPOCHS-1]

means_fcni = last_accs_only.mean(axis=0)
stds_fcni = last_accs_only.std(axis=0)

print("Mean over 10 rounds changing the Number of hidden layers and number of units per hidden layer of the FCN subnet : ")
print(means_fcni)

print("Standard deviation over 10 rounds changing the Number of hidden layers and number of units per hidden layer of the FCN subnet : ")
print(stds_fcni)

raveled_i_max = np.argmax(means_fcni)

unraveled_i_max = np.unravel_index(raveled_i_max, means_fcni.shape)

best_nb_hidden_inner = config.NB_LAYERS[unraveled_i_max[0]]
best_hidden_layer_inner = config.FCNEURONS[unraveled_i_max[1]]
best_acc = np.max(means_fcni)

print("Best score with number if hidden layer {0} and units per hidden layer {1} with accuracy {2}".format(best_nb_hidden_inner, best_hidden_layer_inner, best_acc))

Mean over 10 rounds changing the Number of hidden layers and number of units per hidden layer of the FCN subnet : 
[[0.8703 0.8719 0.8817 0.8863 0.8874]
 [0.8968 0.9078 0.9105 0.908  0.9096]]
Standard deviation over 10 rounds changing the Number of hidden layers and number of units per hidden layer of the FCN subnet : 
[[0.00732189 0.01441839 0.00600083 0.00752396 0.00915642]
 [0.0066453  0.00810925 0.00531507 0.01057355 0.0048621 ]]
Best score with number if hidden layer 2 and units per hidden layer 128 with accuracy 0.9105000000000001


# FCNEURONS (outer network) search

In [8]:
cuda = torch.cuda.is_available()
if cuda:
    print("CUDA available")
else:
    print("NO CUDA")

round_results_nb_neurons_outer = [] #3D

for i in range(rounds):
    results = [] #training_losses, training_acc, test_losses, test_acc
    
    print('round {0} start'.format(i+1))
    
    for ind1, nb in enumerate(config.NB_LAYERS):
        results_tmp = []
        
        for ind2, n in enumerate(config.FCNEURONS):
        
            subnet = FCN(nb_hidden_layers = best_nb_hidden_inner, hidden_layer = best_hidden_layer_inner)

            model = SiameseNet(subnet, nb_hidden_layers = nb, hidden_layer = n)

            training_losses, training_acc, _, _, test_losses, test_acc, _, _ = train_siamese(
                model = model, dataloader = train_dataloader, test_dataloader = test_dataloader,
                epochs = config.EPOCHS, learning_rate = config.LEARNING_RATE, aux_loss = True,
                alpha = config.ALPHA)

            print('{0}/{1}'.format(ind1 * len(config.FCNEURONS) + ind2 + 1, len(config.NB_LAYERS) * len(config.FCNEURONS)))
            print('With parameters nb_hid_inner/hid_inner/nb_hid_out/hid_outer/alpha : {0}/{1}/{2}/{3}/{4}'.format(best_nb_hidden_inner,
                                                                                                                   best_hidden_layer_inner,
                                                                                                                   nb,
                                                                                                                   n,
                                                                                                                   config.ALPHA))
            final_test_loss, final_test_loss_acc = test_losses[-1], test_acc[-1]
            print("On the test set we obtain a loss of {:.2f} and an accuracy of {:.2f}".format(final_test_loss,final_test_loss_acc))

            results_tmp.append([training_losses, training_acc, test_losses, test_acc])
    
        results.append(results_tmp)
    
    print('round {0} end'.format(i+1))
    round_results_nb_neurons_outer.append(results)

CUDA available
round 1 start
1/10
With parameters nb_hid_inner/hid_inner/nb_hid_out/hid_outer/alpha : 2/128/1/32/0.5
On the test set we obtain a loss of 0.34 and an accuracy of 0.89
2/10
With parameters nb_hid_inner/hid_inner/nb_hid_out/hid_outer/alpha : 2/128/1/64/0.5
On the test set we obtain a loss of 0.33 and an accuracy of 0.89
3/10
With parameters nb_hid_inner/hid_inner/nb_hid_out/hid_outer/alpha : 2/128/1/128/0.5
On the test set we obtain a loss of 0.35 and an accuracy of 0.91
4/10
With parameters nb_hid_inner/hid_inner/nb_hid_out/hid_outer/alpha : 2/128/1/256/0.5
On the test set we obtain a loss of 0.32 and an accuracy of 0.90
5/10
With parameters nb_hid_inner/hid_inner/nb_hid_out/hid_outer/alpha : 2/128/1/512/0.5
On the test set we obtain a loss of 0.28 and an accuracy of 0.93
6/10
With parameters nb_hid_inner/hid_inner/nb_hid_out/hid_outer/alpha : 2/128/2/32/0.5
On the test set we obtain a loss of 0.38 and an accuracy of 0.89
7/10
With parameters nb_hid_inner/hid_inner/nb_hid

In [9]:
np.savez("results-fcn-nb_fcneurons_outer_search",round_results_nb_neurons_outer)

copy_of = np.array(round_results_nb_neurons_outer).copy()

last_accs_only = copy_of[:, :, :, 3, config.EPOCHS-1]

means_fcno = last_accs_only.mean(axis=0)
stds_fcno = last_accs_only.std(axis=0)

print("Mean over 10 rounds changing the Number of hidden layers and number of units per hidden layer of the Siamese net : ")
print(means_fcno)

print("Standard deviation over 10 rounds changing the Number of hidden layers and number of units per hidden layer of the Siamese net : ")
print(stds_fcno)

raveled_i_max = np.argmax(means_fcno)

unraveled_i_max = np.unravel_index(raveled_i_max, means_fcno.shape)

best_nb_hidden_outer = config.NB_LAYERS[unraveled_i_max[0]]
best_hidden_layer_outer = config.FCNEURONS[unraveled_i_max[1]]
best_acc = np.max(means_fcno)

print("Best score with number of hidden layer {0} and units per hidden layer {1} with accuracy {2}".format(best_nb_hidden_outer, best_hidden_layer_outer, best_acc))

Mean over 10 rounds changing the Number of hidden layers and number of units per hidden layer of the Siamese net : 
[[0.8893 0.8953 0.9014 0.9055 0.9152]
 [0.8895 0.8997 0.9097 0.9205 0.9204]]
Standard deviation over 10 rounds changing the Number of hidden layers and number of units per hidden layer of the Siamese net : 
[[0.00679779 0.00511957 0.00569561 0.00809012 0.00574108]
 [0.00516236 0.00523546 0.00605062 0.00471699 0.0052    ]]
Best score with number of hidden layer 2 and units per hidden layer 256 with accuracy 0.9205


# ALPHA search

In [10]:
cuda = torch.cuda.is_available()
if cuda:
    print("CUDA available")
else:
    print("NO CUDA")

round_results_alpha = [] #3D

for i in range(rounds):
    results = [] #training_losses, training_acc, test_losses, test_acc
    
    print('round {0} start'.format(i+1))
    
    for ind, a in enumerate(config.ALPHAS):
        
        subnet = FCN(nb_hidden_layers = best_nb_hidden_inner, hidden_layer = best_hidden_layer_inner)

        model = model = SiameseNet(subnet, nb_hidden_layers = best_nb_hidden_outer, hidden_layer = best_hidden_layer_outer)
        
        training_losses, training_acc, _, _, test_losses, test_acc, _, _ = train_siamese(
            model = model, dataloader = train_dataloader, test_dataloader = test_dataloader,
            epochs = config.EPOCHS, learning_rate = config.LEARNING_RATE, aux_loss = True, 
            alpha = a)
        
        print('{0}/{1}'.format(ind+1, len(config.ALPHAS)))
        print('With parameters nb_hid_inner/hid_inner/nb_hid_out/hid_outer/alpha : {0}/{1}/{2}/{3}/{4}'.format(best_nb_hidden_inner,
                                                                                                                   best_hidden_layer_inner,
                                                                                                                   best_nb_hidden_outer,
                                                                                                                   best_hidden_layer_outer,
                                                                                                                   round(a, 2)))
        final_test_loss, final_test_loss_acc = test_losses[-1], test_acc[-1]
        print("On the test set we obtain a loss of {:.2f} and an accuracy of {:.2f}".format(final_test_loss,final_test_loss_acc))
        
        results.append([training_losses, training_acc, test_losses, test_acc])
    
    
    print('round {0} end'.format(i+1))
    round_results_alpha.append(results)

CUDA available
round 1 start
1/10
With parameters nb_hid_inner/hid_inner/nb_hid_out/hid_outer/alpha : 2/128/2/256/0.0
On the test set we obtain a loss of 0.32 and an accuracy of 0.52
2/10
With parameters nb_hid_inner/hid_inner/nb_hid_out/hid_outer/alpha : 2/128/2/256/0.11
On the test set we obtain a loss of 0.28 and an accuracy of 0.92
3/10
With parameters nb_hid_inner/hid_inner/nb_hid_out/hid_outer/alpha : 2/128/2/256/0.22
On the test set we obtain a loss of 0.32 and an accuracy of 0.93
4/10
With parameters nb_hid_inner/hid_inner/nb_hid_out/hid_outer/alpha : 2/128/2/256/0.33
On the test set we obtain a loss of 0.31 and an accuracy of 0.91
5/10
With parameters nb_hid_inner/hid_inner/nb_hid_out/hid_outer/alpha : 2/128/2/256/0.44
On the test set we obtain a loss of 0.29 and an accuracy of 0.93
6/10
With parameters nb_hid_inner/hid_inner/nb_hid_out/hid_outer/alpha : 2/128/2/256/0.56
On the test set we obtain a loss of 0.31 and an accuracy of 0.92
7/10
With parameters nb_hid_inner/hid_inne

In [11]:
np.savez("results-fcn-alpha",round_results_alpha)

copy_of = np.array(round_results_alpha).copy()

last_accs_only = copy_of[:, :, 3, config.EPOCHS-1]

means_alpha = last_accs_only.mean(axis=0)
stds_alpha = last_accs_only.std(axis=0)

print("Mean over 10 rounds changing alpha of the aux loss : ")
print(means_alpha)

print("Standard deviation over 10 rounds changing alpha of the aux loss : ")
print(stds_alpha)

i_max = np.argmax(means_alpha)

best_alpha = config.ALPHAS[i_max]

best_acc = np.max(means_alpha)

print("Best score with alpha of {0} with accuracy {1}".format(best_alpha, best_acc))

Mean over 10 rounds changing alpha of the aux loss : 
[0.5389 0.9282 0.9225 0.9211 0.9189 0.9138 0.9089 0.9015 0.889  0.8406]
Standard deviation over 10 rounds changing alpha of the aux loss : 
[0.05742029 0.00723602 0.00502494 0.0067     0.00620403 0.00752064
 0.00391024 0.0057836  0.00581378 0.00688767]
Best score with alpha of 0.1111111111111111 with accuracy 0.9282
